Here K and tau_i are being calculated using genetic algorithm:

In [ ]:
!pip install control

Define the given second order transfer function

In [ ]:
import control as ctrl

num = [1]
den = [20, 10, 1]

# Defining the second order plant transfer function
plant_tf = ctrl.TransferFunction(num, den)

print("Plant Transfer Function:")
print(plant_tf)

Plant Transfer Function:
<TransferFunction>: sys[6]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


        1
-----------------
20 s^2 + 10 s + 1



Now we define a time vector from 0 to 30 :

In [ ]:
import numpy as np

# Define the time vector
t = np.arange(0, 30.01, 0.01)

print("Time vector:", t)

Time vector: [0.000e+00 1.000e-02 2.000e-02 ... 2.998e+01 2.999e+01 3.000e+01]


Now we create an unit step input:

In [ ]:
r = np.ones_like(t)
print("Step input array ", r)

Step input array  [1. 1. 1. ... 1. 1. 1.]


Definitions of the three errors: ITAE, IAE, and ISE:

In [ ]:
def compute_ITAE(K, plant_tf, r, t):
    # Simulate the closed-loop system
    controller_tf = ctrl.TransferFunction([K[0]*K[1], K[0]], [K[1], 0])
    closed_loop_tf = ctrl.feedback(ctrl.series(controller_tf, plant_tf), 1)
    t_sim, y = ctrl.step_response(closed_loop_tf, T=t)

    # Compute ITAE criteria
    error = r - y
    ITAE_value = np.trapz(t_sim * np.abs(error), t_sim)
    return ITAE_value

def compute_IAE(K,plant_tf,r,t):
    # Simulate the closed-loop system
    controller_tf = ctrl.TransferFunction([K[0]*K[1], K[0]], [K[1], 0])
    closed_loop_tf = ctrl.feedback(ctrl.series(controller_tf, plant_tf), 1)
    t_sim, y = ctrl.step_response(closed_loop_tf, T=t)

    # Compute IAE criteria
    error = r - y
    IAE_value = np.trapz(np.abs(error), t_sim)
    return IAE_value

def compute_ISE(K,plant_tf,r,t):
    # Simulate the closed-loop system
    controller_tf = ctrl.TransferFunction([K[0]*K[1], K[0]], [K[1], 0])
    closed_loop_tf = ctrl.feedback(ctrl.series(controller_tf, plant_tf), 1)
    t_sim, y = ctrl.step_response(closed_loop_tf, T=t)

    # Compute ISE criteria
    error = r - y
    ITAE_value = np.trapz(np.abs(error)**2, t_sim)
    return ITAE_value



In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


Now we use deap to solve the problem with the following parameters:

*   Population size = 125
*   Number of generations = 75
*   cross-over probability = 0.35
*   mutation probability = 0.30

In [ ]:
import numpy as np
from deap import creator, base, tools, algorithms

# Define the objective function for GA:
def ITAE(K):
    return compute_ITAE(K, plant_tf, r, t),

# Create the deap framework objects
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform,-150,150)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)  # Assuming 2 parameters
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)  # Gaussian mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection
toolbox.register("evaluate", ITAE)

# Genetic Algorithm parameters
population_size = 125
num_generations = 75

def main():
    # Create the initial population
    population = toolbox.population(n=population_size)

    # Run the genetic algorithm
    algorithms.eaSimple(population, toolbox, cxpb=0.35, mutpb=0.30, ngen=num_generations, verbose=True)

    # Select the best individual
    best_individual = tools.selBest(population, k=1)[0]

    # Evaluate the best individual to obtain its ITAE value
    best_ITAE_value = ITAE(best_individual)[0]

    return best_individual, best_ITAE_value

if __name__ == "__main__":
    best_solution, best_ITAE_value = main()
    print("Best PID parameters:", best_solution)
    print("Optimal ITAE value:", best_ITAE_value)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	125   
1  	64    
2  	73    
3  	66    
4  	74    
5  	76    
6  	69    
7  	77    
8  	57    
9  	59    
10 	71    
11 	79    
12 	66    
13 	49    
14 	68    
15 	77    
16 	59    
17 	75    
18 	74    
19 	69    
20 	60    
21 	65    
22 	72    
23 	63    
24 	64    
25 	62    
26 	60    
27 	76    
28 	80    
29 	54    
30 	68    
31 	68    
32 	71    
33 	75    
34 	69    
35 	50    
36 	62    
37 	72    
38 	74    
39 	73    
40 	66    
41 	75    
42 	68    
43 	75    
44 	65    
45 	70    
46 	72    
47 	66    
48 	69    
49 	70    
50 	53    
51 	83    
52 	65    
53 	63    
54 	61    
55 	55    
56 	65    
57 	72    
58 	64    
59 	59    
60 	75    
61 	62    
62 	72    
63 	65    
64 	69    
65 	75    
66 	66    
67 	71    
68 	51    
69 	61    
70 	60    
71 	60    
72 	73    
73 	66    
74 	63    
75 	75    
Best PID parameters: [287.66216535391, 236.618106832159]
Optimal ITAE value: 10.732239112221965


We get :
*   K = 287.6621
*   tau_i = 236.61
*   ITAE = 10.73

In [ ]:
import numpy as np
from deap import creator, base, tools, algorithms

def IAE(K):
    return compute_IAE(K, plant_tf, r, t),

# Create the deap framework objects
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform,-150,150)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)  # Gaussian mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection
toolbox.register("evaluate", IAE)

# Genetic Algorithm parameters
population_size = 125
num_generations = 75

def main():
    # Create the initial population
    population = toolbox.population(n=population_size)

    # Run the genetic algorithm
    algorithms.eaSimple(population, toolbox, cxpb=0.35, mutpb=0.30, ngen=num_generations, verbose=True)

    # Select the best individual
    best_individual = tools.selBest(population, k=1)[0]

    # Evaluate the best individual to obtain its IAE value
    best_IAE_value = IAE(best_individual)[0]

    return best_individual, best_IAE_value

if __name__ == "__main__":
    best_solution, best_IAE_value = main()
    print("Best PID parameters:", best_solution)
    print("Optimal IAE value:", best_IAE_value)


gen	nevals
0  	125   
1  	63    
2  	70    
3  	62    
4  	72    
5  	70    
6  	75    
7  	76    
8  	67    
9  	75    
10 	61    
11 	66    
12 	76    
13 	61    
14 	60    
15 	69    
16 	74    
17 	60    
18 	60    
19 	62    
20 	63    
21 	68    
22 	67    
23 	53    
24 	79    
25 	71    
26 	68    
27 	65    
28 	66    
29 	69    
30 	71    
31 	75    
32 	77    
33 	68    
34 	59    
35 	66    
36 	65    
37 	56    
38 	71    
39 	70    
40 	68    
41 	71    
42 	74    
43 	53    
44 	73    
45 	61    
46 	60    
47 	75    
48 	68    
49 	73    
50 	72    
51 	58    
52 	76    
53 	74    
54 	70    
55 	66    
56 	63    
57 	59    
58 	70    
59 	63    
60 	59    
61 	70    
62 	69    
63 	71    
64 	77    
65 	69    
66 	76    
67 	62    
68 	60    
69 	75    
70 	62    
71 	56    
72 	60    
73 	81    
74 	56    
75 	70    
Best PID parameters: [224.42976040627246, -45.648134318835915]
Optimal IAE value: 2.5407169709903776


We get :
*   K = 224.4298
*   tau_i = -45.6481
*   ITAE = 2.5407

In [ ]:
import numpy as np
from deap import creator, base, tools, algorithms

def ISE(K):
    return compute_ISE(K, plant_tf, r, t),

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform,-150,150)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)  # Gaussian mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection
toolbox.register("evaluate", IAE)

# Genetic Algorithm parameters
population_size = 125
num_generations = 75

def main():
    # Create the initial population
    population = toolbox.population(n=population_size)

    # Run the genetic algorithm
    algorithms.eaSimple(population, toolbox, cxpb=0.35, mutpb=0.30, ngen=num_generations, verbose=True)

    # Select the best individual
    best_individual = tools.selBest(population, k=1)[0]

    # Evaluate the best individual to obtain its ISE value
    best_ISE_value = ISE(best_individual)[0]

    return best_individual, best_IAE_value

if __name__ == "__main__":
    best_solution, best_ISE_value = main()
    print("Best PID parameters:", best_solution)
    print("Optimal IAE value:", best_ISE_value)


gen	nevals
0  	125   
1  	59    
2  	60    
3  	61    
4  	72    
5  	68    
6  	75    
7  	76    
8  	69    
9  	66    
10 	65    
11 	72    
12 	78    
13 	78    
14 	72    
15 	65    
16 	54    
17 	65    
18 	66    
19 	68    
20 	64    
21 	57    
22 	69    
23 	75    
24 	76    
25 	78    
26 	72    
27 	64    
28 	64    
29 	73    
30 	78    
31 	69    
32 	65    
33 	64    
34 	61    
35 	73    
36 	57    
37 	62    
38 	48    
39 	60    
40 	66    
41 	77    
42 	69    
43 	72    
44 	77    
45 	73    
46 	73    
47 	60    
48 	62    
49 	65    
50 	67    
51 	68    
52 	62    
53 	60    
54 	60    
55 	51    
56 	73    
57 	69    
58 	62    
59 	74    
60 	69    
61 	69    
62 	64    
63 	73    
64 	64    
65 	72    
66 	72    
67 	67    
68 	62    
69 	76    
70 	64    
71 	69    
72 	70    
73 	74    
74 	64    
75 	68    
Best PID parameters: [197.95551638177366, -51.15790768512122]
Optimal IAE value: 2.5407169709903776


We get :
*   K = 197.9555
*   tau_i = -51.1579
*   ITAE = 2.5407